# Code Completion Parser

SGLang support code fim completion in completion interface for code model. Code fim completion is feature that predicts and inserts missing code snippets within the context of existing code, boosting productivity and reducing errors. 

## Supported Models

Currently, SGLang supports the following code models:
- DeepSeek Coder
- Qwen Coder
- Star Coder

## Usage

### Launching the Server

Specify the `--completion-template` option.

In [ ]:
from openai import OpenAI
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/deepseek-coder-1.3b-base --completion-template deepseek_coder --port 30020 --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

### OpenAI Compatible API

completion template currently have three options: deepseek_coder, qwen_coder, star_ coder

- `completion-template`: the completion template you want to use. 

In [ ]:
# Initialize OpenAI-like client
client = OpenAI(api_key="None", base_url=f"http://0.0.0.0:{port}/v1")
model_name = client.models.list().data[0].id

prompt = "function sum(a: number, b: number): number {"
suffix = "}"

#### Request
it's the same for Stream request and Non-Stream request, so we take non-stream as an example.

In [ ]:
response_non_stream = client.completions.create(
    model=model_name,
    prompt=prompt,
    suffix=suffix,
    temperature=0.3,
    top_p=0.95,
    stream=False,  # Non-streaming
)

print_highlight("==== [FIM] ====")
print_highlight(response_non_stream.choices[0].text)

In [ ]:
terminate_process(server_process)

### Offline Engine API

In [ ]:
import sglang as sgl
from sglang.srt.code_completion_parser import generate_completion_prompt

llm = sgl.Engine(model_path="deepseek-ai/deepseek-coder-1.3b-base")
input = generate_completion_prompt(prompt, suffix, "deepseek_coder")


sampling_params = {
    "max_new_tokens": 50,
    "skip_special_tokens": False,
    "temperature": 0.3,
    "top_p": 0.95,
}
result = llm.generate(prompt=input, sampling_params=sampling_params)

fim_code = result["text"]  # Assume there is only one prompt

print_highlight(fim_code)

In [ ]:
llm.shutdown()